In [1]:
# !pip install praw
# !pip install pyarrow

In [2]:
import praw
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import zstandard as zstd
from datetime import datetime

### Methodology 1. Reddit praw api

In [3]:
# ### put your personal info 
# reddit = praw.Reddit(client_id='', \
#                      client_secret='', \
#                      user_agent='')

In [4]:
# posts = []
# ml_subreddit = reddit.subreddit('depression')
# for post in ml_subreddit.new(limit=1000):
#     posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
# posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
# posts

In [5]:
# posts = []
# ml_subreddit = reddit.subreddit('depression')
# for post in ml_subreddit.hot(limit=1000):
#     posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
# posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])

In [6]:
# posts

In [7]:
# posts.title[0]

In [8]:
# posts.body[0]

### Methodology 2. Reddit archive data dump (https://the-eye.eu/redarcs/) 

In [9]:
ls

depressed_submissions.zst
reddit_data_preprocessing_AnxietyDepression.ipynb
reddit_data_preprocessing_depressed.ipynb
reddit_data_preprocessing_technology.ipynb
technology_submissions.zst
venv/


In [10]:
## step1. load downloaded submission reddit dump 

In [26]:
technology_submissions =\
    pd.read_json('technology_submissions.zst',compression=dict(method='zstd', max_window_size=2147483648),\
    lines = True, nrows = 600000)

In [27]:
# convert the timestamp to utc
technology_submissions['created_utc'] =\
    technology_submissions['created_utc'].apply(lambda x : datetime.utcfromtimestamp(x))

In [13]:
# technology_submissions = technology_submissions.drop(['variants'],axis=1)

In [29]:
# technology_submissions = technology_submissions[['archived', 'author', 'author_flair_background_color', 'author_flair_css_class', 'author_flair_text', 'author_flair_text_color', 'brand_safe', 'can_gild', 'contest_mode', 'created_utc', 'distinguished', 'domain', 'edited', 'gilded', 'hidden', 'hide_score', 'id', 'is_crosspostable', 'is_reddit_media_domain', 'is_self', 'is_video', 'link_flair_css_class', 'link_flair_richtext', 'link_flair_text', 'link_flair_text_color', 'link_flair_type', 'locked', 'no_follow', 'num_comments', 'num_crossposts', 'over_18', 'parent_whitelist_status', 'permalink', 'retrieved_on', 'rte_mode', 'score', 'secure_media', 'secure_media_embed', 'selftext', 'send_replies', 'spoiler', 'stickied', 'subreddit', 'subreddit_id', 'subreddit_name_prefixed', 'subreddit_type', 'suggested_sort', 'thumbnail', 'thumbnail_height', 'thumbnail_width', 'title', 'url', 'whitelist_status', 'author_flair_richtext', 'author_flair_type', 'author_cakeday', 'post_hint', 'preview',
#        'link_flair_background_color', 'link_flair_template_id', 'is_original_content', 'downs', 'num_reports', 'banned_by', 'name', 'likes', 'clicked', 'saved', 'ups', 'approved_by', 'selftext_html', 'created']]

In [14]:
## step2. download the file into parquet file to speed up data load later stage

In [33]:
technology_submissions.to_parquet('technology_submissions.parquet', engine='pyarrow')

In [16]:
## step3. read as parquet file 

In [34]:
technology_submissions = pd.read_parquet('technology_submissions.parquet',engine='pyarrow')
technology_submissions

In [19]:
## step4. feature selection : select only necessary columns

In [35]:
technology_submissions =\
    technology_submissions[['subreddit','author','over_18','link_flair_text','title','selftext','url','created_utc']]
technology_submissions


,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
0,technology,[deleted],False,None,Toyota’s Eyeball Camera To Save Sleepy Drivers,[deleted],http://uk.gizmodo.com/2008/01/25/toyotas_eyeba...,2008-01-25 10:24:23
1,technology,[deleted],False,None,"Evergreen's Goth Keyboard for ""meat-loving peo...",[deleted],http://www.engadget.com/2008/01/25/evergreens-...,2008-01-25 10:56:30
2,technology,ScrewDriver,False,None,"Acoustic ""Invisibility"" Cloaks Possible",,http://news.nationalgeographic.com/news/2008/0...,2008-01-25 12:41:11
3,technology,[deleted],False,None,How to Tell If You Are Addicted to Technology,[deleted],http://www.livescience.com/technology/080125-t...,2008-01-25 22:04:35
4,technology,[deleted],False,None,Microsoft: Vista's not as insecure as XP. Plea...,,http://arstechnica.com/news.ars/post/20080126-...,2008-01-26 15:17:26
...,...,...,...,...,...,...,...,...
599995,technology,sherryreem,False,None,iPhone 5 Nano SIM Cutter: Best Way To Convert ...,,http://areacellphone.com/2012/09/iphone-5-nano...,2012-09-17 14:56:39
599996,technology,[deleted],False,None,Coding win,,http://imgur.com/JP203,2012-09-17 14:54:47
599997,technology,ywalter16,False,None,Mobile Advertising,,http://miabusinessplan.com/mobile-advertising....,2012-09-17 14:54:27
599998,technology,Thund3rbolt,False,None,In 2020 We Can Wear Sony Computers On Our Wrists.,,http://imgur.com/a/XpziA#0,2012-09-17 14:54:05


In [36]:
## step5. basic pre-processing 

In [39]:
# remove blank body posts
technology_submissions = technology_submissions[technology_submissions['selftext'].apply(lambda x : len(x)!=0)]
technology_submissions

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
121075,technology,bloosteak,False,None,Why won't Dlink tech support let me RMA this b...,They've made me try to do an incremental firmw...,https://www.reddit.com/r/technology/comments/8...,2009-06-18 22:07:57
121294,technology,RugerRedhawk,False,None,Fellow Chrome users: Does clicking the pink 'T...,"If I click the Pink text in the top right, it ...",https://www.reddit.com/r/technology/comments/8...,2009-06-19 16:49:54
121305,technology,GoreDragon,False,None,if you want an iPhone today you will have to p...,"So if the phone is $199, the original unsubsid...",https://www.reddit.com/r/technology/comments/8...,2009-06-19 17:27:16
121530,technology,robotlou,False,None,Open Video Conference: Live from NYU June 20th,Great discussions from NYU on the future of on...,https://www.reddit.com/r/technology/comments/8...,2009-06-20 14:49:21
121549,technology,designstrike,False,None,12 High Quality Business XHTML Templates,12 of the most professional looking and high q...,https://www.reddit.com/r/technology/comments/8...,2009-06-20 16:06:38
...,...,...,...,...,...,...,...,...
376673,technology,[deleted],False,None,We need the community to become a TEAM against...,"We've written congress, we've signed petitions...",http://www.reddit.com/r/technology/comments/nz...,2012-01-02 00:58:57
376681,technology,Llort2,False,None,can a video file be compressed by only overwri...,"if so, which formats do this.\n\nhow would I g...",http://www.reddit.com/r/technology/comments/nz...,2012-01-02 00:14:44
476679,technology,TheSkyNet,False,None,"Hi /r/Technology, shall we have a talk about C...","Ladies, gentlemen and variations thereupon, I ...",http://www.reddit.com/r/technology/comments/sx...,2012-04-29 08:21:14
481015,technology,[deleted],False,None,Can you change the graphics card in a Dell Ins...,"Everything else is fine with this thing, excep...",http://www.reddit.com/r/technology/comments/t6...,2012-05-04 15:56:43


In [40]:
## opt out '[deleted]','[removed]' only selftext 
opt_out_list = ['[deleted]','[removed]']
technology_submissions[technology_submissions['selftext'].isin(opt_out_list)]

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc


In [41]:
technology_submissions = technology_submissions[~technology_submissions['selftext'].isin(opt_out_list)]
print(len(technology_submissions))
technology_submissions

8051


,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
121075,technology,bloosteak,False,None,Why won't Dlink tech support let me RMA this b...,They've made me try to do an incremental firmw...,https://www.reddit.com/r/technology/comments/8...,2009-06-18 22:07:57
121294,technology,RugerRedhawk,False,None,Fellow Chrome users: Does clicking the pink 'T...,"If I click the Pink text in the top right, it ...",https://www.reddit.com/r/technology/comments/8...,2009-06-19 16:49:54
121305,technology,GoreDragon,False,None,if you want an iPhone today you will have to p...,"So if the phone is $199, the original unsubsid...",https://www.reddit.com/r/technology/comments/8...,2009-06-19 17:27:16
121530,technology,robotlou,False,None,Open Video Conference: Live from NYU June 20th,Great discussions from NYU on the future of on...,https://www.reddit.com/r/technology/comments/8...,2009-06-20 14:49:21
121549,technology,designstrike,False,None,12 High Quality Business XHTML Templates,12 of the most professional looking and high q...,https://www.reddit.com/r/technology/comments/8...,2009-06-20 16:06:38
...,...,...,...,...,...,...,...,...
376673,technology,[deleted],False,None,We need the community to become a TEAM against...,"We've written congress, we've signed petitions...",http://www.reddit.com/r/technology/comments/nz...,2012-01-02 00:58:57
376681,technology,Llort2,False,None,can a video file be compressed by only overwri...,"if so, which formats do this.\n\nhow would I g...",http://www.reddit.com/r/technology/comments/nz...,2012-01-02 00:14:44
476679,technology,TheSkyNet,False,None,"Hi /r/Technology, shall we have a talk about C...","Ladies, gentlemen and variations thereupon, I ...",http://www.reddit.com/r/technology/comments/sx...,2012-04-29 08:21:14
481015,technology,[deleted],False,None,Can you change the graphics card in a Dell Ins...,"Everything else is fine with this thing, excep...",http://www.reddit.com/r/technology/comments/t6...,2012-05-04 15:56:43


In [23]:
# step6. final data quality check 

In [42]:
print(f'Author : {technology_submissions.iloc[[0],:].author.tolist()[0]}')
print(f'Subreddit : {technology_submissions.iloc[[0],:].subreddit.tolist()[0]}')
print(f'Teenager(True/False) : {technology_submissions.iloc[[0],:].over_18.tolist()[0]}')
print(f'Posted : {technology_submissions.iloc[[0],:].created_utc.tolist()[0]}')
print()
print(f'Title : {technology_submissions.iloc[[0],:].title.tolist()[0]}')
print(f'Body : {technology_submissions.iloc[[0],:].selftext.tolist()[0]}')
print()
print()
print()
print(f'Author : {technology_submissions.iloc[[1],:].author.tolist()[0]}')
print(f'Subreddit : {technology_submissions.iloc[[1],:].subreddit.tolist()[0]}')
print(f'Teenager(True/False) : {technology_submissions.iloc[[1],:].over_18.tolist()[0]}')
print(f'Posted : {technology_submissions.iloc[[1],:].created_utc.tolist()[0]}')
print()
print(f'Title : {technology_submissions.iloc[[1],:].title.tolist()[0]}')
print(f'Body : {technology_submissions.iloc[[1],:].selftext.tolist()[0]}')


Author : bloosteak
Subreddit : technology
Teenager(True/False) : False
Posted : 2009-06-18 22:07:57

Title : Why won't Dlink tech support let me RMA this broken router? They've wasted at least 3.5 hours of my time already making me do pointless things. Three different people now. They're obviously trying to get me to give up. WTF?
Body : They've made me try to do an incremental firmware upgrade which according to firmware notes won't do a thing. MTU size from 1500 to 1400 (won't do a damn thing), even change the IP of my router from 192.160.0.1 to 192.160.10.1 (WTF?) The problem is my router ALWAYS stops working 20-30 minutes after dling/uling torrents. It used to work fine, but it hasn't for months. I wish I could go straight through American Express but I'm still within the 1 year Dlink warranty. I've tried turning on and off QoS, resetting to factory defaults, nothing. The fact that they are stalling pisses me off. I am never ordering another Dlink. 



Author : RugerRedhawk
Subredd

In [43]:
# step7. save final pre-processed data into parquet
technology_submissions.to_parquet('technology_submissions_preprocessed.parquet', engine='pyarrow')
technology_submissions_preprocessed = pd.read_parquet('technology_submissions_preprocessed.parquet',engine='pyarrow')
technology_submissions_preprocessed

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
121075,technology,bloosteak,False,None,Why won't Dlink tech support let me RMA this b...,They've made me try to do an incremental firmw...,https://www.reddit.com/r/technology/comments/8...,2009-06-18 22:07:57
121294,technology,RugerRedhawk,False,None,Fellow Chrome users: Does clicking the pink 'T...,"If I click the Pink text in the top right, it ...",https://www.reddit.com/r/technology/comments/8...,2009-06-19 16:49:54
121305,technology,GoreDragon,False,None,if you want an iPhone today you will have to p...,"So if the phone is $199, the original unsubsid...",https://www.reddit.com/r/technology/comments/8...,2009-06-19 17:27:16
121530,technology,robotlou,False,None,Open Video Conference: Live from NYU June 20th,Great discussions from NYU on the future of on...,https://www.reddit.com/r/technology/comments/8...,2009-06-20 14:49:21
121549,technology,designstrike,False,None,12 High Quality Business XHTML Templates,12 of the most professional looking and high q...,https://www.reddit.com/r/technology/comments/8...,2009-06-20 16:06:38
...,...,...,...,...,...,...,...,...
376673,technology,[deleted],False,None,We need the community to become a TEAM against...,"We've written congress, we've signed petitions...",http://www.reddit.com/r/technology/comments/nz...,2012-01-02 00:58:57
376681,technology,Llort2,False,None,can a video file be compressed by only overwri...,"if so, which formats do this.\n\nhow would I g...",http://www.reddit.com/r/technology/comments/nz...,2012-01-02 00:14:44
476679,technology,TheSkyNet,False,None,"Hi /r/Technology, shall we have a talk about C...","Ladies, gentlemen and variations thereupon, I ...",http://www.reddit.com/r/technology/comments/sx...,2012-04-29 08:21:14
481015,technology,[deleted],False,None,Can you change the graphics card in a Dell Ins...,"Everything else is fine with this thing, excep...",http://www.reddit.com/r/technology/comments/t6...,2012-05-04 15:56:43
